# Multi-layer Forward-feed NN 

## 0. Finish Facial Recognition from last time

## 1. Learn to pickle a dictionary

## 2. Alak -- Checker and Chess Meet with Go (棋)
- ### Explain the rules
- ### Simulate a game

In [4]:
%matplotlib inline
# All imports

from __future__ import print_function, division
from random import choice
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import time
from IPython import display

import sklearn
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.cross_validation import train_test_split

from time import time
from sklearn.metrics import classification_report, confusion_matrix

# -------------> New import <------------------

import pickle
from colorama import Fore, Back, Style
from pdb import set_trace
import re

np.set_printoptions(formatter={'float': '{:.5f}'.format})


## Simulate Alak:
   ### a) Randomly generate the next legal move
   ### b) Decide if any pieces need to be removed from the board
   ### c) The opposite side do step a), then b), etc.
   ### d) Repeat until one side wins (the other side should have zero piece on the board)
   ### Save all the steps of the simulated game in a pickle file (save one side as 'x', the other side as 'o', and unoccupied as '\_' -- thus each step will be saved as a string.  The key should be the step number)


In [59]:
"""DON'T CHANGE ANYTHING"""

# class Alak:
#     def __init__(self):
#         self.step = []
#         self.board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])
#         self.x_pos = np.where(self.board == 'x')[0]
#         self.o_pos = np.where(self.board == 'o')[0]
#         self.random_move = np.random.choice(np.where(self.board == '_')[0])
        
#     def next_move(self):
#         self.random_move = np.random.choice(np.where(self.board == '_')[0])
        
    
def next_move(board, piece):
    move = np.random.choice(np.where(board == '_')[0])
    
    piece_pos = np.where(board == str(piece))[0]    #Indices of x/o pieces
    rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new position, move
    
    board[move], board[rand_piece] = board[rand_piece], board[move]     #Assign piece to move
    
    step = "{}: {} --> {}".format(piece, rand_piece, move)     #Assigns step of piece to new position
    return board, step

def check_dead(board, piece):
    if piece == 'x':
        dead_pat = re.search('xo+x', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('xo+x', ''.join(board))
        dead_pat = re.search('ox+o', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('ox+o', ''.join(board))
        return board
    else:
        dead_pat = re.search('ox+o', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('ox+o', ''.join(board))
        dead_pat = re.search('xo+x', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('xo+x', ''.join(board))
        return board
    
def testing():
    passed = 0
    failed = 0
    special_cases = np.array([list('xx_xox_ooo'), list('_xox_xx_oo'), list('_xoox_ox__'),\
                     list('xoxoxx_oo_'), list('xooox__oxx'), list('xoooox__xx')])
    expected_results = np.array([list('xx_x_x_ooo'), list('_x_x_xx_oo'), list('_x__x_ox__'),\
                     list('x_x_xxooo_'), list('x___x__oxx'), list('x____x__xx')])
    for i, j in enumerate(special_cases):
        try:
            assert np.array_equal(check_dead(special_cases[i], 'x'), expected_results[i])
        except:
            failed += 1
        else:
            passed += 1
    if failed != 0:
        print("{} out of 6 tests passed!".format(passed))
        print(Fore.RED + "{} out of 6 tests failed!".format(failed))
    else:
        print("{} out of 6 tests passed!".format(passed))
    print(Style.RESET_ALL)
    

steps = []
board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])
print(*board)
print()
# board = np.array(['x', '_', 'o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
i = 1


# while 'x' in board and 'o' in board:
while len(np.unique(board)) > 2:
    print("Round: {}\n".format(i))
    print()
############################################
    if len(np.unique(board)) > 2:
        if (board == 'x').sum() > 1:
            board, step = next_move(board, 'x')     #Determines which piece to move
            steps.append(step)
            print(step)
            print(*board)
            board_up = check_dead(board, 'x')
            if board_up != None:
                board = board_up
            print(*board)
        else:
            print("Game over, only one 'x' left.")
            break
#         print()
############################################
    if len(np.unique(board)) > 2:
        if (board == 'o').sum() > 1:
            board, step = next_move(board, 'o')
            steps.append(step)
            print(step)
            print(*board)
            board_up = check_dead(board, 'o')
            if board_up != None:
                board = board_up
            print(*board)
        else:
            print("Game over, only one 'o' left.")
            break
#         print()
    
    i += 1

x x x x _ _ o o o o

Round: 1


x: 2 --> 4
x x _ x x _ o o o o
x x _ x x _ o o o o
o: 9 --> 5
x x _ x x o o o o _
x x _ x x o o o o _
Round: 2


x: 4 --> 2
x x x x _ o o o o _
x x x x _ o o o o _
o: 6 --> 9
x x x x _ o _ o o o
x x x x _ o _ o o o
Round: 3


x: 1 --> 6
x _ x x _ o x o o o
x _ x x _ o _ o o o
o: 9 --> 6
x _ x x _ o o o o _
x _ x x _ o o o o _
Round: 4


x: 2 --> 1
x x _ x _ o o o o _
x x _ x _ o o o o _
o: 7 --> 9
x x _ x _ o o _ o o
x x _ x _ o o _ o o
Round: 5


x: 3 --> 4
x x _ _ x o o _ o o
x x _ _ x o o _ o o
o: 6 --> 2
x x o _ x o _ _ o o
x x o _ x o _ _ o o
Round: 6


x: 0 --> 7
_ x o _ x o _ x o o
_ x o _ x o _ x o o
o: 5 --> 6
_ x o _ x _ o x o o
_ x o _ x _ o _ o o
Round: 7


x: 1 --> 5
_ _ o _ x x o _ o o
_ _ o _ x x o _ o o
o: 6 --> 1
_ o o _ x x _ _ o o
_ o o _ x x _ _ o o
Round: 8


x: 5 --> 0
x o o _ x _ _ _ o o
x o o _ x _ _ _ o o
o: 9 --> 7
x o o _ x _ _ o o _
x o o _ x _ _ o o _
Round: 9


x: 4 --> 5
x o o _ _ x _ o o _
x o o _ _ x _ o o _
o: 8 --> 4
x 

/Users/labuser/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:96: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/Users/labuser/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:111: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [60]:
testing()
print('hello')

5 out of 6 tests passed!
1 out of 6 tests failed!

hello


In [ ]:
from pdb import set_trace
import re
class Alak:
    def __init__(self):
        self.step = []
        self.board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])
        
    def next_move(self, piece):
        move = np.random.choice(np.where(self.board == '_')[0])
    
        piece_pos = np.where(self.board == str(piece))[0]    #Indices of x/o pieces
        rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new position, move

        self.board[move], self.board[rand_piece] = self.board[rand_piece], self.board[move]     #Assign piece to move

        step = "{}: {} --> {}".format(piece, rand_piece, move)     #Assigns step of piece to new position
        self.step.append(step)
        
    
# def next_move(board, piece):
#     move = np.random.choice(np.where(board == '_')[0])
    
#     piece_pos = np.where(board == str(piece))[0]    #Indices of x/o pieces
#     rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new position, move
    
#     board[move], board[rand_piece] = board[rand_piece], board[move]     #Assign piece to move
    
#     step = "{}: {} --> {}".format(piece, rand_piece, move)     #Assigns step of piece to new position
#     return board, step

def check_dead(board, piece):
    if piece == 'x':
        dead_pat = re.search('xo+x', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('xo+x', ''.join(board))
        dead_pat = re.search('ox+o', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('ox+o', ''.join(board))
        return board
    else:
        dead_pat = re.search('ox+o', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('ox+o', ''.join(board))
        dead_pat = re.search('xo+x', ''.join(board))
        while dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board))]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            dead_pat = re.search('xo+x', ''.join(board))
        return board

steps = []
board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])
print(*board)
print()
# board = np.array(['x', '_', 'o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
i = 1


# while 'x' in board and 'o' in board:
while len(np.unique(board)) > 2:
    print("Round: {}\n".format(i))
    print()
############################################
    if len(np.unique(board)) > 2:
        if (board == 'x').sum() > 1:
            board, step = next_move(board, 'x')     #Determines which piece to move
            steps.append(step)
            print(step)
            print(*board)
            board_up = check_dead(board, 'x')
            if board_up != None:
                board = board_up
            print(*board)
        else:
            print("Game over, only one 'x' left.")
            break
#         print()
############################################
    if len(np.unique(board)) > 2:
        if (board == 'o').sum() > 1:
            board, step = next_move(board, 'o')
            steps.append(step)
            print(step)
            print(*board)
            board_up = check_dead(board, 'o')
            if board_up != None:
                board = board_up
            print(*board)
        else:
            print("Game over, only one 'o' left.")
            break
#         print()
    
    i += 1

# End of Week11-1

In [ ]:
test_board = np.array(['o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
stuff = re.search('ox+o', ''.join(test_board))
new_test_board = ''.join(test_board)
# print(np.where(new_test_board == stuff.group().rsplit()))
print(new_test_board)
print(stuff.group())

In [ ]:
test_board = np.array(['x', 'x', '_', 'x', 'o', 'o', 'x', '_', '_', 'o'])
test_board_j = ''.join(test_board)
stuff = re.search('xox', ''.join(test_board))
if stuff != None:
    stuff = stuff.group()
    se = [(m.start(0), m.end(0)) for m in re.finditer(stuff, test_board_j)]
    for elem in se:
        print(test_board)
        np.put(test_board, range(elem[0]+1, elem[1]-1), '_')
        print(test_board)
        
    
# stuff_new = np.array(list(stuff.group()))
# print(np.where(test_board == stuff_new))

In [ ]:
import regex as re

pattern = 'oxo'
string = 'oxoxxo'
se = [(m.start(0), m.end(0)) for m in re.finditer(pattern, string, overlapped=True)]
print(se)
# for elem in se:
    

In [ ]:
stuff = np.array(['x', 'o', 'x', '_'])
print(np.unique(stuff))
cool = np.array(['_', 'o', 'x'])
print(np.unique(stuff) == cool)

In [ ]:
new_board = np.array(['x', '_', 'o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
print(check_dead(new_board, 'o'))

In [ ]:
stuff_board = np.array(['o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
print(re.search('oxo', ''.join(stuff_board)).group())

In [ ]:
stuff_board = np.array(['o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
dead_pat = re.search('ox+o', ''.join(stuff_board))
while dead_pat != None:
    dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(stuff_board))]
    for elem in dead_se:
        np.put(stuff_board, range(elem[0]+1, elem[1]-1), '_')
    dead_pat = re.search('ox+o', ''.join(stuff_board))
print(stuff_board)

In [3]:
test_board = np.array(['o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
print(check_dead(test_board, 'o'))

['o' '_' 'o' '_' 'o' 'o' '_' '_']


In [10]:
print(list('stuff'))

['s', 't', 'u', 'f', 'f']
